In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException

In [13]:
# Setup Selenium Chrome Web Driver
chromedriver = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(chromedriver)
#WebPage="http://www.marathonguide.com/results/browse.cfm?MIDD=480181209" #marathon main page
WebPage="http://www.marathonguide.com/races/races.cfm" #marathon main page


In [14]:
# Navigate in Chrome to specified page.
driver.get(WebPage)
URL = driver.current_url
print(URL) #URL being accessed

http://www.marathonguide.com/races/races.cfm


In [15]:
#get list of drop down
html=urlopen(URL) # Opens URL for html format
soup = BeautifulSoup(html, 'lxml') #creates BeautifulSoup variable

In [40]:
for child in soup.find_all('table')[7].children:
    for td in child:
        print(td)

#date = soup.findAll("td",{"class":"date"}) 
#date.b.text
#option_text=option.text
#option_str=StringIO(option_text)
#dropdown_values = pd.read_csv(option_str, sep="/t", engine = 'python') # data frame of dropdown values



<td colspan="2"><img alt="Race Comments" border="0" height="10" src="../images/Comments_sym.gif" width="15"/>Comments | <img alt="Race News" border="0" height="10" src="../images/News_sym.gif" width="15"/>News |  <img alt="Marathon Press Releases" border="0" height="10" src="../images/Press_sym.gif" width="15"/>Press Releases | <img alt="Marathon Course Maps" border="0" height="10" src="../images/Maps_sym.gif" width="15"/>Course Maps |  <img alt="Race Results" border="0" height="10" src="../images/results_sym.gif" width="15"/> Results</td>
<td align="right" colspan="2"><a href="races.cfm?Sort=RaceDate&amp;Place=USA&amp;StartDate=10%2F1%2F21"><img align="right" alt="Previous Marathons" border="0" height="12" hspace="5" src="../images/smallarrow_left.gif" vspace="2" width="22"/></a>More Marathon Dates:<a href="races.cfm?Sort=RaceDate&amp;Place=USA&amp;StartDate=2%2F3%2F22"><br clear="all"/>
<img align="right" alt="Future Marathons" border="0" height="12" hspace="5" src="../images/small

In [8]:
final=pd.DataFrame()
#for i in range(0,1): #can use to test what raw data looks like to correct header length
for i in range(0,len(dropdown_values['Overall Results'])):   #loop through dropdown values (0 to 'end of dropdown') 
   # if the page looks like it is buffering you can stop the kernel and change to start of the loop to the next value. If stuck buffering on 55, change 0 to 56 and re-run!
    while True:
        try:
            driver.get(WebPage)  
            select=Select(WebDriverWait(driver,5).until(lambda x: x.find_element_by_name('RaceRange')))  #Select dropdown
            select.select_by_visible_text(dropdown_values['Overall Results'][i]) #choose a dropdown value
            python_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > table:nth-child(121) > tbody > tr:nth-child(1) > td:nth-child(2) > table:nth-child(4) > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr > td > table:nth-child(3) > tbody > tr > td:nth-child(1) > form > p:nth-child(6) > input[type=submit]:nth-child(3)")))
            python_button.click()
            break
        except TimeoutException:
            print("Loading took too much time")
            driver.close()
            time.sleep(5)
            chromedriver = "/usr/local/bin/chromedriver"
            driver = webdriver.Chrome(chromedriver)
    
    for element in driver.find_elements_by_css_selector('body > table:nth-child(120) > tbody > tr:nth-child(1) > td:nth-child(2) > table:nth-child(4) > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr > td > table'):
            print(i)    
            rawdata = element.text #<- needs a function to turn text into dataframe(could be an issue after first iteration that you need to use append to add data to the dataframe)
            text_lines = rawdata.split('\n')
            if i == 0:
                head = pd.DataFrame(text_lines[0:8]) #only save the header for the first page, so far every header seems to be the first 8 lines
                del text_lines[0:8]
                data = pd.DataFrame(text_lines)
                final = final.append(data,ignore_index=True)
            else:
                del text_lines[0:9]
                data = pd.DataFrame(text_lines)
                final = final.append(data,ignore_index=True)

            

 

Loading took too much time
Loading took too much time


WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.45)


In [ ]:
print(final[1200:])

In [ ]:
driver.close()

In [ ]:
#writes the csv
final = head.append(final)
final.to_csv('honolulu18.csv') #change file name for each marathon
